In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [2]:
torch.cuda.set_device(3)

## Train

In [3]:
name = 'train_5c_adj_win'

In [4]:
dls = get_adj_data(512, 128, Meta.splits_stg1, windowed=True)
learn = get_learner(dls, partial(xresnet18, c_in=5))

In [5]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.127035,0.183151,0.956620,0.901292,14:57
1,0.111936,0.175153,0.959089,0.907862,15:02
2,0.105601,0.132701,0.963463,0.922376,14:54
3,0.098545,0.115406,0.964689,0.933783,14:55
4,0.095133,0.105786,0.967933,0.935897,15:01
5,0.091442,0.114593,0.966204,0.928372,14:53
6,0.088794,0.102809,0.969306,0.936673,14:54
7,0.084781,0.091575,0.972239,0.944299,14:49
8,0.076862,0.090756,0.973325,0.945318,14:53
9,0.072780,0.085861,0.974212,0.947100,14:36


In [5]:
learn.dls = get_adj_data(512, 256, Meta.splits_stg1, windowed=True)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.139545,0.159368,0.951688,0.899739,17:02
1,0.113615,0.140667,0.955930,0.925126,17:09
2,0.098023,0.114012,0.967186,0.931237,17:05
3,0.089522,0.106319,0.968952,0.937526,17:07
4,0.084729,0.097392,0.970168,0.941791,17:12
5,0.080703,0.089249,0.972101,0.944961,17:11
6,0.075659,0.086661,0.974053,0.948246,17:08
7,0.072911,0.084328,0.974270,0.949583,17:07
8,0.068476,0.080724,0.975328,0.951149,17:07
9,0.065638,0.075776,0.976309,0.953734,17:08


In [6]:
learn.dls = get_adj_data(256, 384, Meta.splits_stg1, windowed=True, img_dir=path_jpg)
do_fit(learn, 2, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.063251,0.075267,0.976275,0.953734,42:58
1,0.061010,0.072575,0.976996,0.955707,42:39


## Submission

In [5]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [6]:
learn.dls = get_adj_test_data(256, sz=384, tst_dir='tst_jpg', windowed=True)

In [7]:
preds,targs = learn.get_preds()

In [8]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [9]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 5c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.14MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [5]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14335274,
 'totalBytes': 27277209,
 'date': '2020-01-30T16:56:03.437Z',
 'description': 'train_5c_adj_win adjacent 5c',
 'errorDescription': None,
 'fileName': 'train_5c_adj_win.csv',
 'publicScore': '0.50331',
 'privateScore': '0.05985',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14335274/14335274.raw'}